In [1]:
# Libraries
import os
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import gradio as gr

c:\Users\hi\OneDrive\Desktop\Market GenAi\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables 
load_dotenv()


True

In [3]:
# Sample marketing data for context
marketing_data = {
    "last_quarter_engagement": {
        "email_open_rate": "24%",
        "click_through_rate": "3.8%",
        "conversion_rate": "1.2%",
        "social_media_engagement": "12% increase",
        "top_performing_channel": "Email marketing",
        "key_insight": "Personalized subject lines increased open rates by 15%"
    },
    "holiday_season_campaign": {
        "email_open_rate": "32%",
        "click_through_rate": "5.2%",
        "conversion_rate": "2.8%",
        "social_media_engagement": "28% increase",
        "top_performing_channel": "Social media ads",
        "key_insight": "Limited-time offers drove 45% of total conversions"
    },
    "q1_campaigns": {
        "campaign_budget": "$50,000",
        "roi": "3.2x",
        "customer_acquisition_cost": "$45",
        "lifetime_value": "$210"
    }
}

In [4]:


class MarketingChatBot:
    def __init__(self):
        # Initialize the ChatGoogleGenerativeAI model
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",  
            google_api_key=os.getenv("GEMINI_API_KEY"),
            temperature=0.7
        )
        
        # Initialize memory to store conversation history
        self.memory = ConversationBufferMemory()
        
        # Store marketing data
        self.marketing_data = marketing_data

        # Define a prompt template with correct input variables
        self.prompt = PromptTemplate(
            input_variables=["history", "input", "marketing_data"],
            template="""You are a helpful marketing analytics assistant. You help marketers analyze campaign performance,
customer engagement trends, and provide data-driven insights. Use the marketing data and conversation history to maintain context.

Marketing Data:
{marketing_data}

Conversation History:
{history}

Marketer: {input}
Assistant:"""
        )

        # Create a chain with the LLM and Prompt (using LLMChain instead of ConversationChain)
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            verbose=True
        )

    def get_response(self, user_input):
        """Get response from the chatbot"""
        try:
            # Format marketing data as a readable string
            def format_marketing_data(data):
                result = []
                for period, metrics in data.items():
                    result.append(f"{period.replace('_', ' ').title()}:")
                    for key, value in metrics.items():
                        result.append(f"  - {key.replace('_', ' ').title()}: {value}")
                    result.append("")
                return "\n".join(result)
            
            marketing_data_str = format_marketing_data(self.marketing_data)

            # Get the conversation history from memory
            memory_variables = self.memory.load_memory_variables({})
            history = memory_variables.get("history", "")

            # Prepare the inputs for the chain
            inputs = {
                "history": history,
                "input": user_input,
                "marketing_data": marketing_data_str
            }

            # Get the response from the chain
            response = self.chain.invoke(inputs)

            # Save the conversation to memory
            self.memory.save_context({"input": user_input}, {"output": response["text"]})

            # Return the response text
            return response["text"]
        except Exception as e:
            return f"Error: {str(e)}"

# Initialize the chatbot
bot = MarketingChatBot()

C:\Users\hi\AppData\Local\Temp\ipykernel_4260\2722217002.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory()
C:\Users\hi\AppData\Local\Temp\ipykernel_4260\2722217002.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(


In [5]:
def chat_interface(message, history):
    """Gradio chat interface"""

    response = bot.get_response(message)
    
    return response


In [ ]:
def respond(message, chat_history):
    """Function to handle user input and generate response"""
    if not message.strip():
        return "", chat_history
    
    # Get bot response
    bot_message = bot.get_response(message)
    
    # Append to chat history
    chat_history.append((message, bot_message))
    
    # Clear input box
    return "", chat_history

# Custom CSS for better UI
custom_css = """
.gradio-container {
    max-height: none !important;
}
footer {
    display: none !important;
}
#chatbot {
    height: 500px !important;
    max-height: 500px !important;
    overflow-y: auto !important;
}
#input-box {
    max-height: none !important;
}
"""

# Create the custom Gradio interface with submit button
with gr.Blocks(title="Marketing Analytics Chatbot", css=custom_css) as demo:
    gr.Markdown("# 🎯 Marketing Analytics Chatbot")
    gr.Markdown("Ask about campaign performance, engagement trends, and marketing analytics!")
    
    # Create chatbot display
    chatbot = gr.Chatbot(
        label="Conversation",
        height=400,
        show_copy_button=True,
        elem_id="chatbot"
        
    )
    
    # Create input area with submit button
    with gr.Row():
        msg = gr.Textbox(
            label="Your Message",
            placeholder="Type your marketing question here...",
            lines=2,
            max_lines=10,
            scale=8,
            elem_id="input-box",
            submit_btn = True 
        )
        clear_btn = gr.Button("Clear", variant="secondary", scale=1, min_width=100)
    
    # Example questions
    gr.Examples(
        examples=[
            "Summarize last quarter's customer engagement trends",
            "Compare email performance between last quarter and holiday season",
            "What was the ROI for Q1 campaigns?",
            "Suggest improvements for our social media strategy"
        ],
        inputs=msg,
        label="Example Questions (Click to insert)"
    )
    
    # Set up event handlers
    msg.submit(
        respond,
        [msg, chatbot],
        [msg, chatbot]
    )
    
    
    clear_btn.click(
        lambda: [],
        None,
        chatbot,
        queue=False
    )

# Run the application
if __name__ == "__main__":
    # Test the chatbot
    print("Testing Marketing ChatBot...")
    test_queries = [
        "Summarize last quarter's customer engagement trends",
        "Now compare with the holiday season campaign"
    ]
    
    for query in test_queries:
        print(f"User: {query}")
        response = bot.get_response(query)
        print(f"Bot: {response}\n")
    
    # Launch the interface
    demo.launch(share=True)

C:\Users\hi\AppData\Local\Temp\ipykernel_4260\337095733.py:39: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Testing Marketing ChatBot...
User: Summarize last quarter's customer engagement trends


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful marketing analytics assistant. You help marketers analyze campaign performance,
customer engagement trends, and provide data-driven insights. Use the marketing data and conversation history to maintain context.

Marketing Data:
Last Quarter Engagement:
  - Email Open Rate: 24%
  - Click Through Rate: 3.8%
  - Conversion Rate: 1.2%
  - Social Media Engagement: 12% increase
  - Top Performing Channel: Email marketing
  - Key Insight: Personalized subject lines increased open rates by 15%

Holiday Season Campaign:
  - Email Open Rate: 32%
  - Click Through Rate: 5.2%
  - Conversion Rate: 2.8%
  - Social Media Engagement: 28% increase
  - Top Performing Channel: Social media ads
  - Key Insight: Limited-time offers drove 45% of total conversions

Q1 Campaigns:
  - Campaign Budget: $50,000
  - Roi: 3.2x
  - Customer Acquisition 



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful marketing analytics assistant. You help marketers analyze campaign performance,
customer engagement trends, and provide data-driven insights. Use the marketing data and conversation history to maintain context.

Marketing Data:
Last Quarter Engagement:
  - Email Open Rate: 24%
  - Click Through Rate: 3.8%
  - Conversion Rate: 1.2%
  - Social Media Engagement: 12% increase
  - Top Performing Channel: Email marketing
  - Key Insight: Personalized subject lines increased open rates by 15%

Holiday Season Campaign:
  - Email Open Rate: 32%
  - Click Through Rate: 5.2%
  - Conversion Rate: 2.8%
  - Social Media Engagement: 28% increase
  - Top Performing Channel: Social media ads
  - Key Insight: Limited-time offers drove 45% of total conversions

Q1 Campaigns:
  - Campaign Budget: $50,000
  - Roi: 3.2x
  - Customer Acquisition Cost: $45
  - Lifetime Value: $210


Conversation History:
Human: Summarize last quarte